# 🎵 Exemplo: Processar "É o gás - pagode.mp3"

## 📊 Análise Detectada:
- **SNR:** 8.3 dB (ruído muito alto)
- **LUFS:** -38.8 (volume muito baixo)
- **Gênero:** Pagode

## ⚙️ Configuração Otimizada:
- Noise reduction: 0.6 (moderado-alto)
- Bass boost: +1.0 dB (surdo, baixo)
- Presence boost: +2.0 dB (vocal)
- Treble boost: +1.5 dB (pandeiro, agogô)

---

## 1. Instalação

In [ ]:
%%capture
!pip install -q librosa soundfile scipy matplotlib numpy noisereduce
print("✓ Dependências instaladas")

## 2. Setup

In [ ]:
from google.colab import drive
import torch

# Montar Drive
drive.mount('/content/drive')

# Verificar GPU
if torch.cuda.is_available():
    print(f"✓ GPU: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️ Sem GPU (ok para processamento sem Demucs)")

## 3. Baixar Pipeline

In [ ]:
!rm -rf /content/audio-pipeline-repo
!git clone -b claude/audio-restoration-pipeline-gAFxk \
  https://github.com/guitorte/musicas.git \
  /content/audio-pipeline-repo

print("✓ Pipeline baixado (versão V3 FIXED)")

## 4. Importar e Carregar Configuração Otimizada

In [ ]:
import sys
sys.path.insert(0, '/content/audio-pipeline-repo/audio-restoration-pipeline')

from modules import AudioRestorationPipeline

# Carregar configuração otimizada para pagode
exec(open('/content/audio-pipeline-repo/audio-restoration-pipeline/config_pagode_optimized.py').read())

print("✓ Pipeline e configurações carregadas")
print("\n📋 Configurações disponíveis:")
print("  • CONFIG_PAGODE_FAST - Rápido, sem Demucs (2-3min) ⭐ RECOMENDADO")
print("  • CONFIG_PAGODE_DEMUCS - Máxima qualidade (15-20min, requer FFmpeg)")
print("  • CONFIG_PAGODE_AGGRESSIVE - Noise reduction muito alto (pode ter artefatos)")

## 5. PROCESSAR (Opção Rápida - SEM Demucs)

In [ ]:
# Inicializar pipeline
pipeline = AudioRestorationPipeline(
    sr=44100,
    output_base_dir='/content/drive/MyDrive/00-restore/restored_output'
)

# Usar configuração otimizada RÁPIDA
CONFIG = CONFIG_PAGODE_FAST  # ⭐ Sem Demucs, 2-3 minutos

print("🎵 Processando com CONFIG_PAGODE_FAST...")
print("⏱️ Tempo estimado: 2-3 minutos")
print("="*70)

# PROCESSAR
result = pipeline.process_audio(
    '/content/drive/MyDrive/00-restore/É o gás - pagode.mp3',
    config=CONFIG
)

print("\n" + "="*70)
print("✓✓✓ SUCESSO! ✓✓✓")
print("="*70)
print(f"\n🎵 Arquivo final: {result['stages']['mastering']['output']}")

## 6. OUVIR RESULTADO

In [ ]:
from IPython.display import Audio, display

if 'result' in globals():
    print("🎧 COMPARAÇÃO: ANTES vs DEPOIS")
    print("="*70)
    
    print("\n🔴 ORIGINAL:")
    display(Audio('/content/drive/MyDrive/00-restore/É o gás - pagode.mp3'))
    
    print("\n🟢 PROCESSADO:")
    display(Audio(result['stages']['mastering']['output']))
    
    print("\n💡 Ouça os dois e compare!")
    print("   • Ruído reduzido")
    print("   • Volume normalizado")
    print("   • Vocal mais presente")
    print("   • Percussão com mais brilho")
else:
    print("⚠️ Execute a célula de processamento primeiro")

## 7. VER ANÁLISE DETALHADA

In [ ]:
import json
from IPython.display import Image

if 'result' in globals():
    # Ler análise
    analysis_path = result['output_dir'] + '/analysis.json'
    with open(analysis_path, 'r') as f:
        analysis = json.load(f)
    
    print("📊 ANÁLISE DO ÁUDIO ORIGINAL")
    print("="*70)
    print(f"Duração: {analysis['duration']:.1f}s")
    print(f"SNR: {analysis['noise_profile']['snr_db']:.1f} dB")
    print(f"LUFS: {analysis['dynamic_range']['lufs_estimate']:.1f}")
    print(f"Peak: {analysis['dynamic_range']['peak_amplitude']:.3f}")
    
    # Visualização
    viz_path = result['output_dir'] + '/analysis_visualization.png'
    print("\n📈 ESPECTROGRAMA:")
    display(Image(filename=viz_path))
else:
    print("⚠️ Execute o processamento primeiro")

## 8. DOWNLOAD DO RESULTADO (Opcional)

In [ ]:
from google.colab import files

if 'result' in globals():
    final_file = result['stages']['mastering']['output']
    print(f"📥 Baixando: {final_file.split('/')[-1]}")
    
    # Descomente para fazer download:
    # files.download(final_file)
    
    print("\n💡 Descomente a linha acima para fazer download")
else:
    print("⚠️ Execute o processamento primeiro")

---

## 🔧 OPÇÃO ALTERNATIVA: Com Demucs (Máxima Qualidade)

Se quiser usar Demucs para máxima qualidade:

### 1. Instalar FFmpeg:
```python
!apt-get update && apt-get install -y ffmpeg
```

### 2. Instalar Demucs:
```python
!pip install -U demucs torchcodec
```

### 3. Usar configuração Demucs:
```python
CONFIG = CONFIG_PAGODE_DEMUCS
result = pipeline.process_audio(
    '/content/drive/MyDrive/00-restore/É o gás - pagode.mp3',
    config=CONFIG
)
```

**Tempo:** 15-20 minutos com GPU

---

## 💡 DICAS:

**Se o resultado ficou com muito ruído ainda:**
```python
CONFIG = CONFIG_PAGODE_AGGRESSIVE  # Noise reduction 0.75
```

**Se o resultado ficou muito "abafado":**
```python
CONFIG = CONFIG_PAGODE_FAST
CONFIG['noise_reduction_strength'] = 0.4  # Reduzir para 0.4
```

**Para ajustar EQ manualmente:**
```python
CONFIG['master_eq']['bass'] = 1.5      # Aumentar bass
CONFIG['master_eq']['presence'] = 2.5  # Aumentar vocal
CONFIG['master_eq']['treble'] = 2.0    # Aumentar brilho
```

---

**Pipeline V3 FIXED - Janeiro 2026**

🎵 Configuração otimizada especificamente para pagode com ruído alto 🎵